# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.90,65,100,7.31,TF,1690661607
1,1,abqaiq,25.9371,49.6776,32.67,30,0,2.14,SA,1690661607
2,2,wailua homesteads,22.0669,-159.3780,27.24,87,75,8.23,US,1690661607
3,3,aasiaat,68.7098,-52.8699,2.87,92,94,2.96,GL,1690661607
4,4,nemuro,43.3236,145.5750,18.06,97,98,2.80,JP,1690661608


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 
                                      'Lat',
                                      geo = True,
                                      size = 'Humidity',
                                      scale = 1,
                                      color = 'City',
                                      alpha = 0.5,
                                      tiles = 'OSM',
                                      frame_width = 700,
                                      frame_height = 500)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
mintemp = 21
maxtemp = 27
windspeed = 4.5
cloudiness = 0

idealweathercondition = city_data_df[
    (city_data_df['Max Temp'] > mintemp) &
    (city_data_df['Max Temp'] < maxtemp) &
    (city_data_df['Wind Speed'] < windspeed) &
    (city_data_df['Cloudiness'] == cloudiness)
]

# Drop any rows with null values
city_data_df=city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.90,65,100,7.31,TF,1690661607
1,1,abqaiq,25.9371,49.6776,32.67,30,0,2.14,SA,1690661607
2,2,wailua homesteads,22.0669,-159.3780,27.24,87,75,8.23,US,1690661607
3,3,aasiaat,68.7098,-52.8699,2.87,92,94,2.96,GL,1690661607
4,4,nemuro,43.3236,145.5750,18.06,97,98,2.80,JP,1690661608


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns= ['City', 'Country', 'Lat','Lng', 'Humidity']

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df=city_data_df[columns].copy()

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity
0,port-aux-francais,TF,-49.3500,70.2167,65
1,abqaiq,SA,25.9371,49.6776,30
2,wailua homesteads,US,22.0669,-159.3780,87
3,aasiaat,GL,68.7098,-52.8699,92
4,nemuro,JP,43.3236,145.5750,97


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
abqaiq - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
aasiaat - nearest hotel: SØMA
nemuro - nearest hotel: イーストハーバーホテル
albany - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
shingu - nearest hotel: ホテルニューパレス
fort bragg - nearest hotel: Airborne Inn Lodging
sechura - nearest hotel: Hospedaje Claritza
adamstown - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
odda - nearest hotel: Hardanger Hotel
falenki - nearest hotel: No hotel found
alofi - nearest hotel: Matavai Resort
buala - nearest hotel: No hotel found
vinita - nearest hotel: No hotel found
ottawa - nearest hotel: Best Western Plus Ottawa Downtown Suites
talnakh - nearest hotel: Талнах
blackmans bay - nearest hotel: Villa Howden
minas de marcona - nearest hotel: No hotel found
college - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,65,Keravel
1,abqaiq,SA,25.9371,49.6776,30,No hotel found
2,wailua homesteads,US,22.0669,-159.3780,87,Hilton Garden Inn Kauai Wailua Bay
3,aasiaat,GL,68.7098,-52.8699,92,SØMA
4,nemuro,JP,43.3236,145.5750,97,イーストハーバーホテル
...,...,...,...,...,...,...
543,waingapu,ID,-9.6567,120.2641,87,HOTEL SANDLEWOOD
544,novyy urgal,RU,51.0736,132.5642,98,No hotel found
545,virginia,US,37.5481,-77.4467,52,Quirk Hotel
546,vakfikebir,TR,41.0458,39.2764,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(hover_cols=["Hotel Name", "Country"],
                                        x="Lng",
                                        y="Lat",
                                      geo = True,
                                      scale = 1,
                                      color = "City",
                                      alpha = 0.5,
                                      tiles = "OSM",
                                      frame_width = 700,
                                      frame_height = 500
                                    )

# Display the map
hotel_map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)